# An introduction to USD basics

The goal of this Colab notebook is to introduce some basics of USD including variants, references, definitions, and schemas. 

While it does not cover the full extent of USD composition, it shows the practical foundations of some of the most useful features to get started with. 

**To run this sample:** click _Runtime_ > _Run all_ from the top menu, or use the <kbd>⌘</kbd>/<kbd>CTRL</kbd>+<kbd>F9</kbd> keyboard shortcut.

## Install the `usd-core` Python package

Install the [`usd-core`](https://pypi.org/project/usd-core/) Python package providing the core USD libraries. Note that it does not provide any of the optional plugins or imaging features from the complete USD distribution.

In [ ]:
! pip install usd-core

# See https://pypi.org/project/usd-core/#history for a list of supported USD
# versions.

## Creating a Layer in USD

USD files represent a layer is the USD lexicon.  From the [Pixar Glossary](https://graphics.pixar.com/usd/docs/USD-Glossary.html#USDGlossary-Layer):
> A *Layer* is the atomic persistent container of scene description for USD.  A layer contains zero or more `PrimSpecs`, that in turn describe `Property` and `Metadata` values.  Each layer possesses an *identifier* that can be used to contruct references to the layer from other layers.  Although it may be possible to someday remove this restriction, layers must currently correspond to files on a filesystem accessible via POSIX filesystem interfaces.

The following section will demonstrate how to create a basic Layer using Python.

See Pixar's [_USD Tutorials_](https://graphics.pixar.com/usd/docs/USD-Tutorials.html) for more details.

In [ ]:
from pxr import Usd, UsdGeom

# Create a tempory stage in memory
stage = Usd.Stage.CreateInMemory('SampleLayer.usda')

# Create a transform and add a sphere as mesh data
xformPrim = UsdGeom.Xform.Define(stage, '/MySphere')

# Set a translation
UsdGeom.XformCommonAPI(xformPrim).SetTranslate((7,8,9))

spherePrim = UsdGeom.Sphere.Define(stage, '/MySphere/MeshData')

# Get the sphere as a generic prim
sphere = stage.GetPrimAtPath('/MySphere/MeshData')

# Get the extent and radius parameters for the prim
radiusAttr = sphere.GetAttribute('radius')
extentAttr = sphere.GetAttribute('extent')

# Access the sphere schema to set the color
colorAttr = spherePrim.GetDisplayColorAttr()

# Set the radius to 2
radiusAttr.Set(2)

# Expand the extents to match the new radius 
extentAttr.Set(extentAttr.Get()*2)

# Make the sphere blue
colorAttr.Set([(0,0,1)])

# Print out the stage
print(stage.GetRootLayer().ExportToString())

# Save the resulting layer
stage.GetRootLayer().Export('SampleLayer.usda')


#usda 1.0

def Xform "MySphere"
{
    double3 xformOp:translate = (7, 8, 9)
    uniform token[] xformOpOrder = ["xformOp:translate"]

    def Sphere "MeshData"
    {
        float3[] extent = [(-2, -2, -2), (2, 2, 2)]
        color3f[] primvars:displayColor = [(0, 0, 1)]
        double radius = 2
    }
}




True

## Understanding how References work

One of the most basic and useful tools for composing scenes in USD is the reference. 

From the [USD Glossary](https://graphics.pixar.com/usd/docs/USD-Glossary.html#USDGlossary-References): 

> The primary use for References is to compose smaller units of scene description into larger *aggregates*, building up a namespace that includes the "encapsulated" result of composing the scene description targeted by a reference.

The following example demonstrates how to make a reference, then override properties of the referenced data. 

In [ ]:
from pxr import Usd, UsdGeom

# Create a tempory stage in memory
stage = Usd.Stage.CreateInMemory('ReferenceExample.usda')

# Create a place for the reference to live
refSphere = stage.OverridePrim('/refSphere')

# Create the reference
refSphere.GetReferences().AddReference('./SampleLayer.usda', '/MySphere')

# Remove the translation operation applied to the base sphere's transform
refXform = UsdGeom.Xformable(refSphere)
refXform.SetXformOpOrder([])

# Print out the stage
print("The Layer\n\n")
print(stage.GetRootLayer().ExportToString())
print("\n\nThe result of Composition \n\n")
print(stage.Flatten().ExportToString())
print("\n\n")
# Override the color of the sphere to be red
overMeshData = UsdGeom.Sphere.Get(stage, '/refSphere/MeshData')
overMeshData.GetDisplayColorAttr().Set([(1,0,0)])

# Print out the stage
print("The Layer\n\n")
print(stage.GetRootLayer().ExportToString())
print("\n\nThe result of Composition \n\n")
print(stage.Flatten().ExportToString())

# Save the resulting layer
stage.GetRootLayer().Export('RefExample.usda')


The Layer


#usda 1.0

over "refSphere" (
    prepend references = @./SampleLayer.usda@</MySphere>
)
{
    uniform token[] xformOpOrder = []
}




The result of Composition 


#usda 1.0
(
    doc = """Generated from Composed Stage of root layer 
"""
)

def Xform "refSphere"
{
    double3 xformOp:translate = (7, 8, 9)
    uniform token[] xformOpOrder = []

    def Sphere "MeshData"
    {
        float3[] extent = [(-2, -2, -2), (2, 2, 2)]
        color3f[] primvars:displayColor = [(0, 0, 1)]
        double radius = 2
    }
}





The Layer


#usda 1.0

over "refSphere" (
    prepend references = @./SampleLayer.usda@</MySphere>
)
{
    uniform token[] xformOpOrder = []

    over "MeshData"
    {
        color3f[] primvars:displayColor = [(1, 0, 0)]
    }
}




The result of Composition 


#usda 1.0
(
    doc = """Generated from Composed Stage of root layer 
"""
)

def Xform "refSphere"
{
    double3 xformOp:translate = (7, 8, 9)
    uniform token[] xformOpOrder = []

    def Sphere "Mesh

True

In [ ]:
from pxr import Usd, UsdGeom

# Create a tempory stage in memory
stage = Usd.Stage.CreateInMemory('VariantExample.usda')

# Create a place for the reference to live
variantSphere = stage.OverridePrim('/variantSphere')

# Create the reference
variantSphere.GetReferences().AddReference('./SampleLayer.usda', '/MySphere')

# Remove the translation operation applied to the base sphere's transform
variantXform = stage.GetPrimAtPath('/variantSphere')

# Clear any color on the base sphere
overMeshData = UsdGeom.Sphere.Get(stage, '/variantSphere/MeshData')
colorAttr = overMeshData.GetDisplayColorAttr()
colorAttr.Clear()

# Add the VariantSet
colorVariants = variantXform.GetVariantSets().AddVariantSet('ColorsRGB')

# Add variants to the VariantSet
colorVariants.AddVariant('red')
colorVariants.AddVariant('green')
colorVariants.AddVariant('blue')

# Set the variant values
colorVariants.SetVariantSelection('red')
with colorVariants.GetVariantEditContext():
    colorAttr.Set([(1,0,0)])

colorVariants.SetVariantSelection('green')
with colorVariants.GetVariantEditContext():
    colorAttr.Set([(0,1,0)])

colorVariants.SetVariantSelection('blue')
with colorVariants.GetVariantEditContext():
    colorAttr.Set([(0,0,1)])

# Set the color to be green
colorVariants.SetVariantSelection('green')

print("\n\n The Layer\n\n")
print(stage.GetRootLayer().ExportToString())
print("\n\n The Result of Composition\n\n")
print(stage.Flatten().ExportToString())

# Save the resulting layer
stage.GetRootLayer().Export('VariantExample.usda')




 The Layer


#usda 1.0

over "variantSphere" (
    prepend references = @./SampleLayer.usda@</MySphere>
    variants = {
        string ColorsRGB = "green"
    }
    prepend variantSets = "ColorsRGB"
)
{
    variantSet "ColorsRGB" = {
        "blue" {
            over "MeshData"
            {
                color3f[] primvars:displayColor = [(0, 0, 1)]
            }

        }
        "green" {
            over "MeshData"
            {
                color3f[] primvars:displayColor = [(0, 1, 0)]
            }

        }
        "red" {
            over "MeshData"
            {
                color3f[] primvars:displayColor = [(1, 0, 0)]
            }

        }
    }
}




 The Result of Composition


#usda 1.0
(
    doc = """Generated from Composed Stage of root layer 
"""
)

def Xform "variantSphere"
{
    double3 xformOp:translate = (7, 8, 9)
    uniform token[] xformOpOrder = ["xformOp:translate"]

    def Sphere "MeshData"
    {
        float3[] extent = [(-2, -2, -2), (2, 2,

True